# Data Wrangling # 1:  MIMIC III Database


## Extract data from MIMIC III Datasets and Organize by Patient. 

## Data Source 

For this project, I used publicly available Electronic Health Records (EHRs) datasets. The MIT Media Lab for Computational Physiology has developed MIMIC-IIIv1.4 dataset based on 46,520 patients who stayed in critical care units of the Beth Israel Deaconess Medical Center of Boston between 2001 and 2012. MIMIC-IIIv1.4 dataset is freely available to researchers across the world. A formal request should be made directly to www.mimic.physionet.org, to gain acess to the data. There is a required course on human research ‘Data or Specimens Only Research’ prior to data acess request. I have secured one here -www.citiprogram.org/verify/?kb6607b78-5821-4de5-8cad-daf929f7fbbf-33486907

The dataset has 26 relational tables including patient’s hospital admission, callout information when patient was ready for discharge, caregiver information, electronic charted events including vital signs and any additional information relevant to patient care, patient demographic data, list of services the patient was admitted or transferred under,  ICU stay types, diagnoses types, laboratory measurments, microbiology tests and sensitivity, prescription data and billing information. 

Although I have full access to the MIMIC-IIIv1.4 datasets, I can not share any part of the data publicly. If you are interested to learn more about the data, there is a MIMIC III Demo dataset based on 100 patients https://mimic.physionet.org/gettingstarted/demo/. If you are interested to requesting access to the data - https://mimic.physionet.org/gettingstarted/access/. 



In [2]:
import os
import pandas as pd
import numpy as np
import random
import sys
import csv

In [3]:
os.getcwd()

'C:\\Users\\abebu\\Google Drive\\Abebual\\Capstone Project 1\\9. MIMIC III\\data_wrangling'

In [3]:
os.chdir("C://Users/abebu/Google Drive/mimic-iii-clinical-database-1.4/Charts")

In [4]:
pt=pd.read_csv('PATIENTS.csv', header=0, index_col=0)
pt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46520 entries, 234 to 31844
Data columns (total 7 columns):
SUBJECT_ID     46520 non-null int64
GENDER         46520 non-null object
DOB            46520 non-null object
DOD            15759 non-null object
DOD_HOSP       9974 non-null object
DOD_SSN        13378 non-null object
EXPIRE_FLAG    46520 non-null int64
dtypes: int64(2), object(5)
memory usage: 2.8+ MB


In [5]:
pt.DOB = pd.to_datetime(pt.DOB)
pt.DOD = pd.to_datetime(pt.DOD)
pt.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
ROW_ID,,,,,,,
234,249,F,2075-03-13,NaT,NaN,NaN,0
235,250,F,2164-12-27,2188-11-22,2188-11-22 00:00:00,NaN,1
236,251,M,2090-03-15,NaT,NaN,NaN,0
237,252,M,2078-03-06,NaT,NaN,NaN,0
238,253,F,2089-11-26,NaT,NaN,NaN,0


In [6]:
adm=pd.read_csv("ADMISSIONS.csv", header=0, index_col=0) 
adm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58976 entries, 21 to 58598
Data columns (total 18 columns):
SUBJECT_ID              58976 non-null int64
HADM_ID                 58976 non-null int64
ADMITTIME               58976 non-null object
DISCHTIME               58976 non-null object
DEATHTIME               5854 non-null object
ADMISSION_TYPE          58976 non-null object
ADMISSION_LOCATION      58976 non-null object
DISCHARGE_LOCATION      58976 non-null object
INSURANCE               58976 non-null object
LANGUAGE                33644 non-null object
RELIGION                58518 non-null object
MARITAL_STATUS          48848 non-null object
ETHNICITY               58976 non-null object
EDREGTIME               30877 non-null object
EDOUTTIME               30877 non-null object
DIAGNOSIS               58951 non-null object
HOSPITAL_EXPIRE_FLAG    58976 non-null int64
HAS_CHARTEVENTS_DATA    58976 non-null int64
dtypes: int64(4), object(14)
memory usage: 8.5+ MB


In [7]:
adm = adm[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ETHNICITY', 'MARITAL_STATUS', 'ADMISSION_TYPE', 'INSURANCE']]
adm.ADMITTIME = pd.to_datetime(adm.ADMITTIME)
adm.DISCHTIME = pd.to_datetime(adm.DISCHTIME)
adm.DEATHTIME = pd.to_datetime(adm.DEATHTIME)
adm.head()

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ETHNICITY,MARITAL_STATUS,ADMISSION_TYPE,INSURANCE
ROW_ID,,,,,,,,,
21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaT,WHITE,MARRIED,EMERGENCY,Private
22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaT,WHITE,MARRIED,ELECTIVE,Medicare
23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaT,WHITE,MARRIED,EMERGENCY,Medicare
24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaT,WHITE,SINGLE,EMERGENCY,Private
25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaT,WHITE,MARRIED,EMERGENCY,Private


In [8]:
adm.HADM_ID.isnull().sum()

0

In [9]:
icu=pd.read_csv("ICUSTAYS.csv", header=0, index_col=0) 
icu.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61532 entries, 365 to 59810
Data columns (total 11 columns):
SUBJECT_ID        61532 non-null int64
HADM_ID           61532 non-null int64
ICUSTAY_ID        61532 non-null int64
DBSOURCE          61532 non-null object
FIRST_CAREUNIT    61532 non-null object
LAST_CAREUNIT     61532 non-null object
FIRST_WARDID      61532 non-null int64
LAST_WARDID       61532 non-null int64
INTIME            61532 non-null object
OUTTIME           61522 non-null object
LOS               61522 non-null float64
dtypes: float64(1), int64(5), object(5)
memory usage: 5.6+ MB


In [10]:
icu.INTIME = pd.to_datetime(icu.INTIME)
icu.OUTTIME = pd.to_datetime(icu.OUTTIME)
icu.ICUSTAY_ID.isnull().sum()

0

In [11]:
icu.HADM_ID.isnull().sum()

0

In [12]:
def subj_merge(table1, table2):
    return table1.merge(table2, how='left', left_on=['SUBJECT_ID'], right_on=['SUBJECT_ID'])

In [13]:
m1=subj_merge(pt, adm)
m1.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ETHNICITY,MARITAL_STATUS,ADMISSION_TYPE,INSURANCE
0,249,F,2075-03-13,NaT,NaN,NaN,0,116935,2149-12-17 20:41:00,2149-12-31 14:55:00,NaT,WHITE,DIVORCED,EMERGENCY,Medicare
1,249,F,2075-03-13,NaT,NaN,NaN,0,149546,2155-02-03 20:16:00,2155-02-14 11:15:00,NaT,WHITE,DIVORCED,EMERGENCY,Medicare
2,249,F,2075-03-13,NaT,NaN,NaN,0,158975,2156-04-27 15:33:00,2156-05-14 15:30:00,NaT,WHITE,DIVORCED,EMERGENCY,Medicare
3,250,F,2164-12-27,2188-11-22,2188-11-22 00:00:00,NaN,1,124271,2188-11-12 09:22:00,2188-11-22 12:00:00,2188-11-22 12:00:00,BLACK/AFRICAN AMERICAN,SINGLE,EMERGENCY,Self Pay
4,251,M,2090-03-15,NaT,NaN,NaN,0,117937,2110-07-27 06:46:00,2110-07-29 15:23:00,NaT,UNKNOWN/NOT SPECIFIED,NaN,EMERGENCY,Private


In [14]:
def subjadm_merge (table1, table2):
    return table1.merge(table2, how='left', left_on=['SUBJECT_ID', 'HADM_ID'], right_on=['SUBJECT_ID', 'HADM_ID'])

In [15]:
m2=subjadm_merge(m1,icu)
m2.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,HADM_ID,ADMITTIME,DISCHTIME,...,INSURANCE,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,249,F,2075-03-13,NaT,NaN,NaN,0,116935,2149-12-17 20:41:00,2149-12-31 14:55:00,...,Medicare,215044.0,carevue,TSICU,MICU,23.0,12.0,2149-12-18 20:06:02,2149-12-24 13:31:45,5.7262
1,249,F,2075-03-13,NaT,NaN,NaN,0,149546,2155-02-03 20:16:00,2155-02-14 11:15:00,...,Medicare,269035.0,metavision,MICU,MICU,50.0,50.0,2155-02-03 20:17:29,2155-02-05 18:34:02,1.9282
2,249,F,2075-03-13,NaT,NaN,NaN,0,149546,2155-02-03 20:16:00,2155-02-14 11:15:00,...,Medicare,263055.0,metavision,TSICU,TSICU,14.0,14.0,2155-02-07 18:51:16,2155-02-11 16:00:39,3.8815
3,249,F,2075-03-13,NaT,NaN,NaN,0,158975,2156-04-27 15:33:00,2156-05-14 15:30:00,...,Medicare,282599.0,metavision,MICU,MICU,52.0,52.0,2156-05-01 18:10:12,2156-05-03 18:43:45,2.0233
4,249,F,2075-03-13,NaT,NaN,NaN,0,158975,2156-04-27 15:33:00,2156-05-14 15:30:00,...,Medicare,263882.0,metavision,MICU,MICU,52.0,52.0,2156-05-10 17:47:35,2156-05-11 19:16:03,1.0614


In [16]:
m2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62722 entries, 0 to 62721
Data columns (total 24 columns):
SUBJECT_ID        62722 non-null int64
GENDER            62722 non-null object
DOB               62722 non-null datetime64[ns]
DOD               24735 non-null datetime64[ns]
DOD_HOSP          16618 non-null object
DOD_SSN           20905 non-null object
EXPIRE_FLAG       62722 non-null int64
HADM_ID           62722 non-null int64
ADMITTIME         62722 non-null datetime64[ns]
DISCHTIME         62722 non-null datetime64[ns]
DEATHTIME         6650 non-null datetime64[ns]
ETHNICITY         62722 non-null object
MARITAL_STATUS    52344 non-null object
ADMISSION_TYPE    62722 non-null object
INSURANCE         62722 non-null object
ICUSTAY_ID        61532 non-null float64
DBSOURCE          61532 non-null object
FIRST_CAREUNIT    61532 non-null object
LAST_CAREUNIT     61532 non-null object
FIRST_WARDID      61532 non-null float64
LAST_WARDID       61532 non-null float64
INTIME      

In [17]:
m2['ADMITTIME'] = pd.to_datetime(m2['ADMITTIME']).dt.normalize()
m2.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,HADM_ID,ADMITTIME,DISCHTIME,...,INSURANCE,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,249,F,2075-03-13,NaT,NaN,NaN,0,116935,2149-12-17,2149-12-31 14:55:00,...,Medicare,215044.0,carevue,TSICU,MICU,23.0,12.0,2149-12-18 20:06:02,2149-12-24 13:31:45,5.7262
1,249,F,2075-03-13,NaT,NaN,NaN,0,149546,2155-02-03,2155-02-14 11:15:00,...,Medicare,269035.0,metavision,MICU,MICU,50.0,50.0,2155-02-03 20:17:29,2155-02-05 18:34:02,1.9282
2,249,F,2075-03-13,NaT,NaN,NaN,0,149546,2155-02-03,2155-02-14 11:15:00,...,Medicare,263055.0,metavision,TSICU,TSICU,14.0,14.0,2155-02-07 18:51:16,2155-02-11 16:00:39,3.8815
3,249,F,2075-03-13,NaT,NaN,NaN,0,158975,2156-04-27,2156-05-14 15:30:00,...,Medicare,282599.0,metavision,MICU,MICU,52.0,52.0,2156-05-01 18:10:12,2156-05-03 18:43:45,2.0233
4,249,F,2075-03-13,NaT,NaN,NaN,0,158975,2156-04-27,2156-05-14 15:30:00,...,Medicare,263882.0,metavision,MICU,MICU,52.0,52.0,2156-05-10 17:47:35,2156-05-11 19:16:03,1.0614


In [18]:
m2.to_csv('m2.csv')

In [19]:
m2_1icu = m2.groupby('ICUSTAY_ID')
m2_1icu.apply(lambda x: x.sample(n=1, replace=True)).head()

m2_5k=m2.sample(n=5000)
m2_5k.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 8249 to 39169
Data columns (total 24 columns):
SUBJECT_ID        5000 non-null int64
GENDER            5000 non-null object
DOB               5000 non-null datetime64[ns]
DOD               2022 non-null datetime64[ns]
DOD_HOSP          1321 non-null object
DOD_SSN           1747 non-null object
EXPIRE_FLAG       5000 non-null int64
HADM_ID           5000 non-null int64
ADMITTIME         5000 non-null datetime64[ns]
DISCHTIME         5000 non-null datetime64[ns]
DEATHTIME         528 non-null datetime64[ns]
ETHNICITY         5000 non-null object
MARITAL_STATUS    4132 non-null object
ADMISSION_TYPE    5000 non-null object
INSURANCE         5000 non-null object
ICUSTAY_ID        4913 non-null float64
DBSOURCE          4913 non-null object
FIRST_CAREUNIT    4913 non-null object
LAST_CAREUNIT     4913 non-null object
FIRST_WARDID      4913 non-null float64
LAST_WARDID       4913 non-null float64
INTIME            4913 non-null

In [20]:
m2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62722 entries, 0 to 62721
Data columns (total 24 columns):
SUBJECT_ID        62722 non-null int64
GENDER            62722 non-null object
DOB               62722 non-null datetime64[ns]
DOD               24735 non-null datetime64[ns]
DOD_HOSP          16618 non-null object
DOD_SSN           20905 non-null object
EXPIRE_FLAG       62722 non-null int64
HADM_ID           62722 non-null int64
ADMITTIME         62722 non-null datetime64[ns]
DISCHTIME         62722 non-null datetime64[ns]
DEATHTIME         6650 non-null datetime64[ns]
ETHNICITY         62722 non-null object
MARITAL_STATUS    52344 non-null object
ADMISSION_TYPE    62722 non-null object
INSURANCE         62722 non-null object
ICUSTAY_ID        61532 non-null float64
DBSOURCE          61532 non-null object
FIRST_CAREUNIT    61532 non-null object
LAST_CAREUNIT     61532 non-null object
FIRST_WARDID      61532 non-null float64
LAST_WARDID       61532 non-null float64
INTIME      

In [21]:
dx=pd.read_csv("DIAGNOSES_ICD.csv", header=0, index_col=0) 
dx_codes=pd.read_csv("D_ICD_DIAGNOSES.csv", header=0, index_col=0)
dx[['SUBJECT_ID', 'HADM_ID', 'SEQ_NUM']] = dx[['SUBJECT_ID', 'HADM_ID', 'SEQ_NUM']]
dx_codes=dx_codes[['ICD9_CODE', 'SHORT_TITLE', 'LONG_TITLE']]
dx=dx.merge(dx_codes, how='left', left_on='ICD9_CODE', right_on='ICD9_CODE')
dx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 651047 entries, 0 to 651046
Data columns (total 6 columns):
SUBJECT_ID     651047 non-null int64
HADM_ID        651047 non-null int64
SEQ_NUM        651000 non-null float64
ICD9_CODE      651000 non-null object
SHORT_TITLE    634709 non-null object
LONG_TITLE     634709 non-null object
dtypes: float64(1), int64(2), object(3)
memory usage: 34.8+ MB


In [22]:
dx.ICD9_CODE.isnull().sum()

47

In [23]:
# drop ICD9_CODE with NULL value
dx.drop_duplicates().set_index('ICD9_CODE')
dx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 651047 entries, 0 to 651046
Data columns (total 6 columns):
SUBJECT_ID     651047 non-null int64
HADM_ID        651047 non-null int64
SEQ_NUM        651000 non-null float64
ICD9_CODE      651000 non-null object
SHORT_TITLE    634709 non-null object
LONG_TITLE     634709 non-null object
dtypes: float64(1), int64(2), object(3)
memory usage: 34.8+ MB


In [24]:
m3=subjadm_merge(m2, dx)
m3.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,HADM_ID,ADMITTIME,DISCHTIME,...,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS,SEQ_NUM,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,249,F,2075-03-13,NaT,NaN,NaN,0,116935,2149-12-17,2149-12-31 14:55:00,...,MICU,23.0,12.0,2149-12-18 20:06:02,2149-12-24 13:31:45,5.7262,1.0,49322,Ch obst asth w (ac) exac,Chronic obstructive asthma with (acute) exacer...
1,249,F,2075-03-13,NaT,NaN,NaN,0,116935,2149-12-17,2149-12-31 14:55:00,...,MICU,23.0,12.0,2149-12-18 20:06:02,2149-12-24 13:31:45,5.7262,2.0,51882,Other pulmonary insuff,"Other pulmonary insufficiency, not elsewhere c..."
2,249,F,2075-03-13,NaT,NaN,NaN,0,116935,2149-12-17,2149-12-31 14:55:00,...,MICU,23.0,12.0,2149-12-18 20:06:02,2149-12-24 13:31:45,5.7262,3.0,41071,"Subendo infarct, initial","Subendocardial infarction, initial episode of ..."
3,249,F,2075-03-13,NaT,NaN,NaN,0,116935,2149-12-17,2149-12-31 14:55:00,...,MICU,23.0,12.0,2149-12-18 20:06:02,2149-12-24 13:31:45,5.7262,4.0,42841,Ac syst/diastol hrt fail,Acute combined systolic and diastolic heart fa...
4,249,F,2075-03-13,NaT,NaN,NaN,0,116935,2149-12-17,2149-12-31 14:55:00,...,MICU,23.0,12.0,2149-12-18 20:06:02,2149-12-24 13:31:45,5.7262,5.0,99812,Hematoma complic proc,Hematoma complicating a procedure


In [25]:
#m3.to_csv('m3.csv')

In [26]:
m3['AGE'] = (m3['INTIME']).dt.date - (m3['DOB']).dt.date
m3['AGE']=pd.to_numeric(m3['AGE'], downcast='integer', errors='ignore')
m3['AGE']=m3['AGE'].astype(str)
m3['AGE'] =m3['AGE'].map(lambda x: x.rstrip(' days, 0:00:00'))

In [27]:
m3['AGE'].isna().sum()

0

In [28]:
m3['AGE']=pd.to_numeric(m3['AGE'], downcast='integer', errors='ignore')
m3.AGE.head()

0    27308
1    27308
2    27308
3    27308
4    27308
Name: AGE, dtype: object

In [29]:
m3.AGE.isnull().sum()

0

In [30]:
m3.to_csv('m3.csv')

In [31]:
m3_kids = m3[m3.AGE<='6570']

In [32]:
m3_adults = m3[m3.AGE>'6570']
m3_adults.info()

In [33]:
kids_sample_id=m3_kids.groupby('SUBJECT_ID').apply(lambda m3_kids : m3_kids.sample(1))
kids_sample_id=kids_sample_id[['SUBJECT_ID']].sample (n=5000, random_state=1)
kids_sample_id.head()

In [34]:
kids_sample_id.set_index('SUBJECT_ID')

In [35]:
kids_id_list=kids_sample_id['SUBJECT_ID'].values.tolist()
kids_id_list

In [36]:
m3_kids.set_index('SUBJECT_ID')

In [37]:
kids_5k=m3_kids[m3_kids['SUBJECT_ID'].isin(kids_id_list)]
kids_5k.info()

In [38]:
kids_5k.to_csv('kids_5k.csv')

In [39]:
adults_sample_id=m3_adults.groupby('SUBJECT_ID').apply(lambda m3_adults : m3_adults.sample(1))
adults_sample_id.head()

In [40]:
adults_sample_id=adults_sample_id[['SUBJECT_ID']]
adults_sample_id.set_index('SUBJECT_ID')

In [41]:
adults_id_list=adults_sample_id['SUBJECT_ID'].values.tolist()
adults_id_list

In [42]:
sample_id_list=kids_id_list + adults_id_list
sample_id_list

In [43]:
def allUnique(x):
    seen = set()
    return not any(i in seen or seen.add(i) for i in x)

In [44]:
allUnique(sample_id_list)

In [45]:
allUnique(kids_id_list)

In [46]:
allUnique(adults_id_list)

In [47]:
set([x for x in sample_id_list if sample_id_list.count(x) > 1])

It looks like the AGE variable is problematic. I will need to check this later. For now i will focus on a general sample of both kids and adults by directly sampling from the m3 dataset, instead of the sub kids and adults datasets.

In [48]:
sample_id=m3.groupby('SUBJECT_ID').apply(lambda m3 : m3.sample(1))
sample_id=sample_id[['SUBJECT_ID']].sample (n=5000, random_state=1)
sample_id.head()

,,SUBJECT_ID
SUBJECT_ID,,
2973,48783,2973
63110,322505,63110
83724,386737,83724
73457,583262,73457
99374,579369,99374


In [49]:
sample_id.set_index('SUBJECT_ID')

""
SUBJECT_ID
2973
63110
83724
73457
99374
...
2872
15430
10394


In [4]:
#id_list=sample_id['SUBJECT_ID'].values.tolist()
#id_list

In [51]:
allUnique(id_list)

True

In [52]:
icu_5k=m3[m3['SUBJECT_ID'].isin(id_list)]
icu_5k.info()

In [53]:
icu_5k.head()

In [54]:
s=pd.read_csv("SERVICES.csv", index_col=0)
s.info()

In [55]:
s = s[['SUBJECT_ID', 'HADM_ID', 'PREV_SERVICE', 'CURR_SERVICE']]
s.head()

In [56]:
s_5k=s[s['SUBJECT_ID'].isin(id_list)]
s_5k.info()

In [57]:
icu_stays=subjadm_merge(icu_5k, s_5k)
icu_stays.info()

In [58]:
icu_stays.to_csv('icu_stays_5k.csv')

### Working Dataset 1: ICU Stays 
icu_stays dataframe or icu_stays_5k.CSV includes an ICU stays of randomly sampled 5000 patients and merged between the following tables:
+ PATIENTS.csv
+ ADMISSIONS.csv
+ SERVICES.csv
+ DIAGNOSES_ICD.csv
+ D_ICD_DIAGNOSES.csv
+ ICUSTAYS

In [59]:
#merge laboratory results table with serv_5k. First lets merge between lab table and lab description table
#chunks = []
#c_size=10000000
#for lab_chunk in pd.read_csv("LABEVENTS.csv", index_col=0, chunksize=c_size):
   # chunks.append(lab_chunk.query("ITEMID>0").copy())
#print(len(chunks))

In [60]:
#lab_chunks=pd.concat(chunks)
#lab_chunks.shape

In [61]:
#lab_chunks.info()

In [62]:
#d_lab=pd.read_csv("D_LABITEMS.csv", index_col=0)
#d_lab.info()

In [4]:
def item_merge (table1, table2):
    return table1.merge(table2, how='left', left_on=['ITEMID'], right_on=['ITEMID'])

In [64]:
#labevents=item_merge(lab_chunks, d_lab)

In [65]:
#labevents.info()

In [66]:
#labevents_5k=labevents[labevents['SUBJECT_ID'].isin(id_list)]
#labevents_5k.info()

Out of the around 28million labratory events, we have got matched [with 5k subject sample] labratory events of 3.1million. 

In [67]:
labevents_5k.to_csv('labevents_5k.csv')

In [ ]:
### Working Dataset 2: Laboratory Events 

labevetns_5k dataframe or labevents_5K.CSV includes a randomly selected 5000 patients laboratory events merged between the following tables:
+ LABEVENTS.csv
+ D_LABITEMS.csv

In [5]:
%%time
c1=pd.read_csv("CHARTEVENTS.csv",index_col=0, nrows=20000000, dtype={'RESULTSTATUS': 'object',
       'STOPPED': 'object',
       'VALUE': 'object'})

C:\Users\abebu\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Wall time: 23.3 s


In [19]:
%%time
c2=pd.read_csv("CHARTEVENTS.csv", names=cols, index_col=0, header=None, skiprows=20000000, nrows=20000000, dtype={'RESULTSTATUS': 'object',
       'STOPPED': 'object',
       'VALUE': 'object'})

Wall time: 28.6 s


In [24]:
%%time
c3=pd.read_csv("CHARTEVENTS.csv", names=cols, index_col=0, header=None, skiprows=40000000, nrows=20000000, dtype={'RESULTSTATUS': 'object',
       'STOPPED': 'object',
       'VALUE': 'object'})

Wall time: 32.8 s


In [29]:
%%time
c4=pd.read_csv("CHARTEVENTS.csv", names=cols, index_col=0, header=None, skiprows=60000000, nrows=40000000, dtype={'RESULTSTATUS': 'object',
       'STOPPED': 'object',
       'VALUE': 'object'})

Wall time: 1min 6s


In [34]:
%%time
c5=pd.read_csv("CHARTEVENTS.csv", names=cols, index_col=0, header=None, skiprows=100000000, nrows=50000000, dtype={'RESULTSTATUS': 'object',
       'STOPPED': 'object',
       'VALUE': 'object'})

Wall time: 1min 43s


In [39]:
%%time
c6=pd.read_csv("CHARTEVENTS.csv", names=cols, index_col=0, header=None, skiprows=150000000, nrows=50000000, dtype={'RESULTSTATUS': 'object',
       'STOPPED': 'object',
       'VALUE': 'object'})

Wall time: 1min 48s


In [74]:
%%time
c7=pd.read_csv("CHARTEVENTS.csv", index_col=0, names=cols, header=None, skiprows=200000000, nrows=50000000, dtype={'RESULTSTATUS': 'object',
       'STOPPED': 'object',
       'VALUE': 'object'})

Wall time: 5min 2s


In [85]:
%%time
c8=pd.read_csv("CHARTEVENTS.csv", index_col=0, names=cols, header=None, skiprows=250000000, nrows=81000000, dtype={'RESULTSTATUS': 'object',
       'STOPPED': 'object',
       'VALUE': 'object'})

<string>:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 8min 10s


In [86]:
list(c1)

['SUBJECT_ID',
 'HADM_ID',
 'ICUSTAY_ID',
 'ITEMID',
 'CHARTTIME',
 'STORETIME',
 'CGID',
 'VALUE',
 'VALUENUM',
 'VALUEUOM',
 'WARNING',
 'ERROR',
 'RESULTSTATUS',
 'STOPPED']

In [6]:
cols= ['SUBJECT_ID',
 'HADM_ID',
 'ICUSTAY_ID',
 'ITEMID',
 'CHARTTIME',
 'STORETIME',
 'CGID',
 'VALUE',
 'VALUENUM',
 'VALUEUOM',
 'WARNING',
 'ERROR',
 'RESULTSTATUS',
 'STOPPED']

In [17]:
ids=pd.read_csv('ids2.csv', header=0, index_col=0)
ids.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11163 entries, 0 to 11162
Data columns (total 3 columns):
SUBJECT_ID    11163 non-null int64
HADM_ID       11163 non-null int64
ICUSTAY_ID    11163 non-null float64
dtypes: float64(1), int64(2)
memory usage: 348.8 KB


In [76]:
c7.shape

(50000000, 14)

In [8]:
c1 = c1.merge(ids, how='inner', left_on= ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'], right_on= ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'])

In [20]:
c2 = c2.merge(ids, how='inner', left_on= ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'], right_on= ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'])

In [25]:
c3 = c3.merge(ids, how='inner', left_on= ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'], right_on= ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'])

In [30]:
c4 = c4.merge(ids, how='inner', left_on= ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'], right_on= ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'])

In [35]:
c5 = c5.merge(ids, how='inner', left_on= ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'], right_on= ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'])

In [40]:
c6 = c6.merge(ids, how='inner', left_on= ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'], right_on= ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'])

In [77]:
c7 = c7.merge(ids, how='inner', left_on= ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'], right_on= ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'])

In [87]:
c8 = c8.merge(ids, how='inner', left_on= ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'], right_on= ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID'])

In [88]:
c8.shape

(29428250, 14)

In [9]:
d_c=pd.read_csv('D_ITEMS.csv', header=0, index_col=0) 
d_c=d_c[['ITEMID', 'LABEL', 'CATEGORY']]
d_c.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12487 entries, 457 to 14522
Data columns (total 3 columns):
ITEMID      12487 non-null int64
LABEL       12483 non-null object
CATEGORY    6049 non-null object
dtypes: int64(1), object(2)
memory usage: 390.2+ KB


In [10]:
c1=item_merge(c1, d_c)

In [21]:
c2=item_merge(c2, d_c)

In [26]:
c3=item_merge(c3, d_c)

In [31]:
c4=item_merge(c4, d_c)

In [36]:
c5=item_merge(c5, d_c)

In [41]:
c6=item_merge(c6, d_c)

In [79]:
c7=item_merge(c7, d_c)

In [89]:
c8=item_merge(c8, d_c)

In [90]:
c8.shape

(29428250, 16)

In [81]:
list(c7)

['SUBJECT_ID',
 'HADM_ID',
 'ICUSTAY_ID',
 'ITEMID',
 'CHARTTIME',
 'STORETIME',
 'CGID',
 'VALUE',
 'VALUENUM',
 'VALUEUOM',
 'WARNING',
 'ERROR',
 'RESULTSTATUS',
 'STOPPED',
 'LABEL',
 'CATEGORY']

In [11]:
labels = ['Arterial Blood Pressure diastolic', 
'Arterial Blood Pressure systolic', 
'Arterial Blood Pressure mean', 
'ART BP Diastolic', 
'ART BP Systolic', 
'ART BP mean',
'Non Invasive Blood Pressure diastolic', 
'Non Invasive Blood Pressure systolic', 
'Non Invasive Blood Pressure mean', 
'Admission Weight (Kg)',
'Admission Weight (lbs.)',
'Anion gap',
'Arterial O2 Saturation',
'BUN',
'Heart Rate',
'Height',
'Height (cm)',
'Hemoglobin',
'O2 saturation pulseoxymetry',
'Platelet Count',
'Potassium (serum)',
'Respiratory Rate',
'Temperature Celsius',
'Temperature Fahrenheit',
'WBC',
'Daily Weight']

In [12]:
c1=c1[c1['LABEL'].isin(labels)]

In [22]:
c2=c2[c2['LABEL'].isin(labels)]

In [27]:
c3=c3[c3['LABEL'].isin(labels)]

In [32]:
c4=c4[c4['LABEL'].isin(labels)]

In [37]:
c5=c5[c5['LABEL'].isin(labels)]

In [42]:
c6=c6[c6['LABEL'].isin(labels)]

In [82]:
c7=c7[c7['LABEL'].isin(labels)]

In [91]:
c8=c8[c8['LABEL'].isin(labels)]

In [13]:
c1.shape

(5419480, 16)

In [14]:
c2.shape

(647323, 16)

In [26]:
c3.shape

(101173, 16)

In [40]:
c4.shape

(242444, 16)

In [52]:
c5.shape

(264272, 16)

In [63]:
c6.shape

(225702, 16)

In [83]:
c7.shape

(0, 16)

In [92]:
c8.shape

(0, 16)

In [41]:
c4.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
0,4908,106923,294026.0,618,2136-07-31 21:00:00,2136-07-31 21:06:00,21205,16,16.0,BPM,NaN,NaN,NaN,NotStopd,Respiratory Rate,NaN
31,4908,106923,294026.0,211,2136-07-31 22:00:00,2136-07-31 23:20:00,21205,86,86.0,BPM,NaN,NaN,NaN,NotStopd,Heart Rate,NaN
54,4908,106923,294026.0,618,2136-07-31 22:00:00,2136-07-31 23:20:00,21205,15,15.0,BPM,NaN,NaN,NaN,NotStopd,Respiratory Rate,NaN
82,4908,106923,294026.0,211,2136-08-01 05:00:00,2136-08-01 05:17:00,21205,84,84.0,BPM,NaN,NaN,NaN,NotStopd,Heart Rate,NaN
96,4908,106923,294026.0,618,2136-08-01 05:00:00,2136-08-01 05:17:00,21205,16,16.0,BPM,NaN,NaN,NaN,NotStopd,Respiratory Rate,NaN


In [14]:
c1=c1[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID', 'CHARTTIME', 'VALUENUM', 'VALUEUOM', 'LABEL', 'CATEGORY']]

In [23]:
c2=c2[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID', 'CHARTTIME', 'VALUENUM', 'VALUEUOM', 'LABEL', 'CATEGORY']]

In [28]:
c3=c3[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID', 'CHARTTIME', 'VALUENUM', 'VALUEUOM', 'LABEL', 'CATEGORY']]

In [33]:
c4=c4[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID', 'CHARTTIME', 'VALUENUM', 'VALUEUOM', 'LABEL', 'CATEGORY']]

In [38]:
c5=c5[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID', 'CHARTTIME', 'VALUENUM', 'VALUEUOM', 'LABEL', 'CATEGORY']]

In [43]:
c6=c6[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID', 'CHARTTIME', 'VALUENUM', 'VALUEUOM', 'LABEL', 'CATEGORY']]

In [65]:
c6

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,VALUENUM,VALUEUOM,LABEL,CATEGORY
2,22930,121115,236659.0,618,2187-11-18 14:00:00,13.0,BPM,Respiratory Rate,NaN
27,22930,121115,236659.0,211,2187-11-16 23:00:00,84.0,BPM,Heart Rate,NaN
45,22930,121115,236659.0,618,2187-11-16 23:00:00,19.0,BPM,Respiratory Rate,NaN
63,22930,121115,236659.0,211,2187-11-17 00:00:00,75.0,BPM,Heart Rate,NaN
68,22930,121115,236659.0,618,2187-11-17 00:00:00,13.0,BPM,Respiratory Rate,NaN
...,...,...,...,...,...,...,...,...,...
4100111,32781,142684,294872.0,618,2134-09-27 08:00:00,24.0,BPM,Respiratory Rate,NaN
4100155,32781,142684,294872.0,211,2134-09-26 11:00:00,90.0,BPM,Heart Rate,NaN
4100192,32781,142684,294872.0,211,2134-09-26 20:00:00,79.0,BPM,Heart Rate,NaN
4100208,32781,142684,294872.0,618,2134-09-26 20:00:00,22.0,BPM,Respiratory Rate,NaN


In [19]:
c1.to_csv('c1.csv')

In [18]:
c2.to_csv('c2.csv')

In [29]:
c3.to_csv('c3.csv')

In [44]:
c4.to_csv('c4.csv')

In [55]:
c5.to_csv('c5.csv')

In [66]:
c6.to_csv('c6.csv')

In [54]:
# Now that the final processed datasets are managable size, lets concat charts dataframes 
frames = [c1, c2, c3, c4, c5, c6]
charts = pd.concat(frames, sort=False)

In [109]:
charts.to_csv('c_c.csv')

In [44]:
c1['LABEL'] = c1['LABEL'].astype('category')

In [ ]:
c1['LABEL'].cat.set

In [45]:
c2['LABEL'] = c2['LABEL'].astype('category')

In [46]:
c3['LABEL'] = c3['LABEL'].astype('category')

In [47]:
c4['LABEL'] = c4['LABEL'].astype('category')

In [48]:
c5['LABEL'] = c5['LABEL'].astype('category')

In [49]:
c6['LABEL'] = c6['LABEL'].astype('category')

In [1]:
#Now lets reshpe the charts dfs using pivot table
p1=c1.pivot_table(values='VALUENUM', index=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID','VALUEUOM'], columns='LABEL')
# the drading memory error - i will continue to work without pivot table, using charts dataset.

### Working Dataset 3: Chart Events 

charts dataframe or c_c.CSV includes a randomly selected 5000 patients charts events merged between the following tables:
+ CHARTEVENTS.csv
+ D_ITEMS.csv

5000 patients sums 6,003,394 chart events. 

In [144]:
micro=pd.read_csv('MICROBIOLOGYEVENTS.csv', header=0, index_col=0) 
micro=micro[['SUBJECT_ID', 'HADM_ID', 'SPEC_ITEMID', 'SPEC_TYPE_DESC', 'ORG_ITEMID', 'ORG_NAME', 'AB_ITEMID', 'AB_NAME', 'DILUTION_VALUE', 'INTERPRETATION']]
micro.SPEC_ITEMID=d_c.ITEMID
micro.ORG_ITEMID=d_c.ITEMID
micro.AB_ITEMID=d_c.ITEMID
micro.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 631726 entries, 744 to 630935
Data columns (total 10 columns):
SUBJECT_ID        631726 non-null int64
HADM_ID           631726 non-null int64
SPEC_ITEMID       12487 non-null float64
SPEC_TYPE_DESC    631726 non-null object
ORG_ITEMID        12487 non-null float64
ORG_NAME          328016 non-null object
AB_ITEMID         12487 non-null float64
AB_NAME           275834 non-null object
DILUTION_VALUE    267178 non-null float64
INTERPRETATION    275834 non-null object
dtypes: float64(4), int64(2), object(4)
memory usage: 53.0+ MB


In [145]:
micro

,SUBJECT_ID,HADM_ID,SPEC_ITEMID,SPEC_TYPE_DESC,ORG_ITEMID,ORG_NAME,AB_ITEMID,AB_NAME,DILUTION_VALUE,INTERPRETATION
ROW_ID,,,,,,,,,,
744,96,170324,825.0,BRONCHOALVEOLAR LAVAGE,825.0,PSEUDOMONAS AERUGINOSA,825.0,NaN,NaN,NaN
745,96,170324,826.0,SPUTUM,826.0,NaN,826.0,NaN,NaN,NaN
746,96,170324,827.0,BLOOD CULTURE,827.0,NaN,827.0,NaN,NaN,NaN
747,96,170324,828.0,BLOOD CULTURE,828.0,NaN,828.0,NaN,NaN,NaN
748,96,170324,829.0,URINE,829.0,NaN,829.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
630931,99783,126090,NaN,TISSUE,NaN,ASPERGILLUS FUMIGATUS,NaN,NaN,NaN,NaN
630932,99783,126090,NaN,TISSUE,NaN,ASPERGILLUS FUMIGATUS,NaN,NaN,NaN,NaN
630933,99783,126090,NaN,TISSUE,NaN,ASPERGILLUS FUMIGATUS,NaN,NaN,NaN,NaN


In [142]:
micro.AB_ITEMID.isnull().sum()

355892

In [157]:
spec=micro[['SUBJECT_ID', 'HADM_ID', 'ITEMID', 'SPEC_TYPE_DESC', 'DILUTION_VALUE', 'INTERPRETATION']]

In [158]:
org=micro[['SUBJECT_ID', 'HADM_ID', 'ORG_ITEMID', 'ORG_NAME', 'DILUTION_VALUE', 'INTERPRETATION']]

In [159]:
ab=micro[['SUBJECT_ID', 'HADM_ID', 'AB_ITEMID', 'AB_NAME', 'DILUTION_VALUE', 'INTERPRETATION']]

In [160]:
org.rename(columns = {'ORG_ITEMID':'ITEMID', 'ORG_NAME':'DESC'}, inplace = True)
ab.rename(columns = {'AB_ITEMID':'ITEMID', 'AB_NAME':'DESC'}, inplace = True)
spec.rename(columns = {'SPEC_TYPE_DESC':'DESC'}, inplace = True)

In [161]:
micro_t=pd.concat([spec, org, ab])

In [162]:
micro_t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1895178 entries, 744 to 630935
Data columns (total 6 columns):
SUBJECT_ID        int64
HADM_ID           int64
ITEMID            float64
DESC              object
DILUTION_VALUE    float64
INTERPRETATION    object
dtypes: float64(2), int64(2), object(2)
memory usage: 101.2+ MB


In [163]:
micro_t.INTERPRETATION.isnull().sum()

1067676

In [164]:
micro_t.head()

,SUBJECT_ID,HADM_ID,ITEMID,DESC,DILUTION_VALUE,INTERPRETATION
ROW_ID,,,,,,
744,96,170324,825.0,BRONCHOALVEOLAR LAVAGE,NaN,NaN
745,96,170324,826.0,SPUTUM,NaN,NaN
746,96,170324,827.0,BLOOD CULTURE,NaN,NaN
747,96,170324,828.0,BLOOD CULTURE,NaN,NaN
748,96,170324,829.0,URINE,NaN,NaN


In [165]:
micro_5k=micro_t[micro_t['SUBJECT_ID'].isin(id_list)]
micro_5k.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212097 entries, 763 to 630903
Data columns (total 6 columns):
SUBJECT_ID        212097 non-null int64
HADM_ID           212097 non-null int64
ITEMID            5118 non-null float64
DESC              137263 non-null object
DILUTION_VALUE    88134 non-null float64
INTERPRETATION    91152 non-null object
dtypes: float64(2), int64(2), object(2)
memory usage: 11.3+ MB


In [166]:
micro_5k=item_merge(micro_5k, d_c)

In [167]:
micro_5k

,SUBJECT_ID,HADM_ID,ITEMID,DESC,DILUTION_VALUE,INTERPRETATION,LABEL,ABBREVIATION,CATEGORY
0,102,195700,3072.0,BLOOD CULTURE - NEONATE,NaN,NaN,IRON 45-160,NaN,NaN
1,52,190797,347.0,BLOOD CULTURE,NaN,NaN,ImpSkin Character #4,NaN,NaN
2,52,190797,348.0,BLOOD CULTURE,NaN,NaN,ImpSkin Character #5,NaN,NaN
3,52,190797,349.0,URINE,NaN,NaN,ImpSkin Character #6,NaN,NaN
4,52,190797,350.0,STOOL,NaN,NaN,ImpSkin #3 [Drain],NaN,NaN
...,...,...,...,...,...,...,...,...,...
212092,99781,147562,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212093,99781,147562,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212094,99781,147562,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212095,99781,147562,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [169]:
micro_5k=micro_5k[['SUBJECT_ID', 'HADM_ID', 'ITEMID', 'LABEL', 'ABBREVIATION', 'CATEGORY', 'DESC', 'DILUTION_VALUE', 'INTERPRETATION']]

In [170]:
micro_5k.head()

,SUBJECT_ID,HADM_ID,ITEMID,LABEL,ABBREVIATION,CATEGORY,DESC,DILUTION_VALUE,INTERPRETATION
0,102,195700,3072.0,IRON 45-160,NaN,NaN,BLOOD CULTURE - NEONATE,NaN,NaN
1,52,190797,347.0,ImpSkin Character #4,NaN,NaN,BLOOD CULTURE,NaN,NaN
2,52,190797,348.0,ImpSkin Character #5,NaN,NaN,BLOOD CULTURE,NaN,NaN
3,52,190797,349.0,ImpSkin Character #6,NaN,NaN,URINE,NaN,NaN
4,52,190797,350.0,ImpSkin #3 [Drain],NaN,NaN,STOOL,NaN,NaN


### Working Dataset 4:microbiology table

+ D_ITEMS.csv
+ MICROBIOLOGYEVENTS.csv

In [189]:
rx=pd.read_csv('PRESCRIPTIONS.csv', header=0, index_col=0) 
rx.STARTDATE = pd.to_datetime(rx.STARTDATE)
rx.ENDDATE = pd.to_datetime(rx.ENDDATE)
rx.info()

C:\Users\abebu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4156450 entries, 2214776 to 3996061
Data columns (total 18 columns):
SUBJECT_ID           int64
HADM_ID              int64
ICUSTAY_ID           float64
STARTDATE            datetime64[ns]
ENDDATE              datetime64[ns]
DRUG_TYPE            object
DRUG                 object
DRUG_NAME_POE        object
DRUG_NAME_GENERIC    object
FORMULARY_DRUG_CD    object
GSN                  object
NDC                  float64
PROD_STRENGTH        object
DOSE_VAL_RX          object
DOSE_UNIT_RX         object
FORM_VAL_DISP        object
FORM_UNIT_DISP       object
ROUTE                object
dtypes: datetime64[ns](2), float64(2), int64(2), object(12)
memory usage: 602.5+ MB


In [190]:
rx

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
ROW_ID,,,,,,,,,,,,,,,,,,
2214776,6,107064,NaN,2175-06-11,2175-06-12,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,4.690617e+08,1mg Capsule,2,mg,2,CAP,PO
2214775,6,107064,NaN,2175-06-11,2175-06-12,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,5.601728e+07,5mg Tablet,5,mg,1,TAB,PO
2215524,6,107064,NaN,2175-06-11,2175-06-12,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,3.380550e+08,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
2216265,6,107064,NaN,2175-06-11,2175-06-12,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.000000e+00,HEPARIN BASE,250,ml,250,ml,IV
2214773,6,107064,NaN,2175-06-11,2175-06-12,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,5.482972e+07,20mg Tablet,20,mg,1,TAB,PO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3996662,98887,121032,238144.0,2144-09-06,2144-09-06,MAIN,PredniSONE,PredniSONE,PredniSONE,PRED20,006751,5.400182e+07,20 mg Tablet,40,mg,2,TAB,PO/NG
3996070,98887,121032,238144.0,2144-09-06,2144-09-06,MAIN,Ipratropium Bromide Neb,Ipratropium Bromide Neb,Ipratropium Bromide Neb,IPRA2H,021700,4.879801e+08,2.5mL Vial,1,NEB,1,VIAL,IH
3996063,98887,121032,238144.0,2144-09-06,2144-09-06,MAIN,HYDROmorphone (Dilaudid),HYDROmorphone (Dilaudid),HYDROmorphone,HYDR20/100NS,048078,6.155302e+10,20 mg / 100 mL Premix Bag,0.12,mg,0.01,BAG,IVPCA


In [191]:
rx_5k=rx[rx['SUBJECT_ID'].isin(id_list)]
rx_5k.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 456351 entries, 616864 to 3488584
Data columns (total 18 columns):
SUBJECT_ID           456351 non-null int64
HADM_ID              456351 non-null int64
ICUSTAY_ID           298758 non-null float64
STARTDATE            456084 non-null datetime64[ns]
ENDDATE              455862 non-null datetime64[ns]
DRUG_TYPE            456351 non-null object
DRUG                 456351 non-null object
DRUG_NAME_POE        272355 non-null object
DRUG_NAME_GENERIC    272511 non-null object
FORMULARY_DRUG_CD    456163 non-null object
GSN                  401523 non-null object
NDC                  455915 non-null float64
PROD_STRENGTH        456221 non-null object
DOSE_VAL_RX          456221 non-null object
DOSE_UNIT_RX         456221 non-null object
FORM_VAL_DISP        456221 non-null object
FORM_UNIT_DISP       456213 non-null object
ROUTE                456233 non-null object
dtypes: datetime64[ns](2), float64(2), int64(2), object(12)
memory usage: 6

In [192]:
rx_5k.to_csv('rx_5k.csv')

### Working Dataset 5:PRESCRIPTIONS

rx_5k dataframe or rx_5K.csv subsets the 5000 randomly selected patients and their medication related order enteries. 
5000 patients covers 456,351 prescription entries out of the total of 4.1million entries. 

+ PRESCRIPTIONS.csv